In [1]:
!pip install pandas
!pip install sklearn
!pip install umap-learn

In [2]:
import torch, baukit
import pandas as pd
import numpy as np
from baukit import pbar
import os
import matplotlib.pyplot as plt

/share/u/koyena/.conda/envs/rome/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: /share/u/koyena/.conda/envs/rome/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [3]:
def compute_euclidean(x, y):
    return np.sqrt(np.sum((x-y)**2))

def get_dists(origin,all_vecs):
    dists = np.empty(len(all_vecs))
    for v in range(len(all_vecs)):
    #for v in pbar(range(len(all_vecs))):
        dist = compute_euclidean(origin,all_vecs[v])
        dists[v] = dist
    return dists

In [4]:
# dimension reduction - pca / umap

In [5]:
token_csv = "../data/tokens.csv"
token_df = pd.read_csv(token_csv)
tokens = token_df['token'].values
token_types = token_df['token_type'].values

In [6]:
npy_datapath = "../data/npy_files"
npy_files = os.listdir(npy_datapath)

In [7]:
def find_k_closest_points(origin_index, all_vecs, tokens, k=1):
    dists = get_dists(all_vecs[origin_index], all_vecs)
    sorted_ind = np.argsort(dists)
    k_counter = 0
    k_points = []
    for index in sorted_ind:
        if index == origin_index:
            continue
        else:
            if (k_counter < k):
                k_points.append([tokens[index], index])
                k_counter += 1
    return k_points


layer_names = []
source_tokens = []
source_token_types = []
closest_tokens = []
closest_token_types = []
top_k_values = []
for curr_file in pbar(npy_files):
    curr_layer_path = npy_datapath + '/'+ curr_file
    curr_layer_mats = np.load(curr_layer_path)
    layer_name = curr_file.split('.')[0]
    for t in range(len(tokens)):
        k_points = find_k_closest_points(t, curr_layer_mats, tokens)
        for i in range(len(k_points)):
            
            layer_names.append(layer_name)
            source_tokens.append(tokens[t])
            source_token_types.append(token_types[t])
            closest_tokens.append(k_points[i][0])
            closest_token_types.append(token_types[k_points[i][1]])
            top_k_values.append(i)

closest_df = pd.DataFrame({"LayerName": layer_names,
                          "SourceToken": source_tokens,
                          "SourceTokenType": source_token_types,
                          "ClosestToken": closest_tokens,
                          "ClosestTokenType": closest_token_types,
                          "TopK": top_k_values})

  0%|          | 0/48 [00:00<?, ?it/s]

In [ ]:
from umap import UMAP
import plotly.express as px

def project_key_vecs_umap(all_vecs, tokens, token_types):
    umap_2d = UMAP(n_components=2, init="random", random_state = 0)
    projections = umap_2d.fit_transform(all_vecs)
    df = pd.DataFrame()
    df["tokens"] = tokens
    df["token_types"] = token_types
    df["comp-1"] = projections[:,0]
    df["comp-2"] = projections[:,1]
    
    fig = px.scatter(
    df, x="comp-1", y="comp-2",
    color="token_types", hover_data=["tokens"],labels={"color": "token_types"})
    fig.show()
    
curr_layer_path = npy_datapath + '/'+ npy_files[0]
curr_layer_mats = np.load(curr_layer_path)
project_key_vecs_umap(curr_layer_mats, tokens, token_types)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
2022-11-02 16:56:34.032231: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-02 16:56:37.123169: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

def project_key_vecs_tsne(all_vecs, tokens, token_types):

#     vec_df = pd.DataFrame({"KeyVec": all_vecs, "Token": tokens, "TokenType": token_types})
    tsne = TSNE(n_components=2, random_state = 0)
    projections = tsne.fit_transform(all_vecs)
    df = pd.DataFrame()
    df["tokens"] = tokens
    df["token_types"] = token_types
    df["comp-1"] = projections[:,0]
    df["comp-2"] = projections[:,1]
    
    fig = px.scatter(
    df, x="comp-1", y="comp-2",
    color="token_types", hover_data=["tokens"],labels={"color": "token_types"})
    fig.show()
    
curr_layer_path = npy_datapath + '/'+ npy_files[0]
curr_layer_mats = np.load(curr_layer_path)
project_key_vecs_tsne(curr_layer_mats, tokens, token_types)

In [ ]:
# Token Type closeness
type_df = closest_df[["LayerName", "SourceTokenType", "ClosestTokenType"]]

layers = []
celeb_to_common_count = []
celeb_to_celeb_count = []
common_to_celeb_count = []
common_to_common_count = []

N = 48
ind = np.arange(N) 
width = 0.25


for i in range(48):
    layer_name = "transformer-h-" + str(i)
    layers.append(str(i))
    celeb_to_common = ((type_df["LayerName"] == layer_name) & 
                       (type_df["SourceTokenType"] == "celeb_people") & 
                       (type_df["ClosestTokenType"] == "common_people"))
    celeb_to_celeb = ((type_df["LayerName"] == layer_name) & 
                      (type_df["SourceTokenType"] == "celeb_people") & 
                      (type_df["ClosestTokenType"] == "celeb_people"))
    common_to_celeb = ((type_df["LayerName"] == layer_name) & 
                       (type_df["SourceTokenType"] == "common_people") & 
                       (type_df["ClosestTokenType"] == "celeb_people"))
    common_to_common = ((type_df["LayerName"] == layer_name) & 
                       (type_df["SourceTokenType"] == "common_people") & 
                       (type_df["ClosestTokenType"] == "common_people"))
    
    celeb_to_common_count.append(len(type_df[celeb_to_common] == True))
    celeb_to_celeb_count.append(len(type_df[celeb_to_celeb] == True))
    common_to_celeb_count.append(len(type_df[common_to_celeb] == True))
    common_to_common_count.append(len(type_df[common_to_common] == True))
    
f = plt.figure()
f.set_figwidth(20)
f.set_figheight(10)

bar1 = plt.bar(ind, celeb_to_common_count, width, color = 'blue')
bar2 = plt.bar(ind+width, celeb_to_celeb_count, width, color='green')
bar3 = plt.bar(ind+width*2, common_to_celeb_count, width, color = 'orange')
bar4 = plt.bar(ind+width*3, common_to_celeb_count, width, color = 'red')

plt.xlabel("Layer")
plt.ylabel('Frequency')
plt.title("Closeness based on Token Type")
  
plt.xticks(ind+width,layers)
plt.legend( (bar1, bar2, bar3, bar4), ('celeb to common', 'celeb to celeb', 'common to celeb', 'common to common'))

plt.show()


In [ ]:
# Token Type closeness
synt_df = closest_df[["LayerName", "SourceToken", "ClosestToken"]]

layers = []
similar_count = []
diff_count = []

N = 48
ind = np.arange(N) 
width = 0.4


for i in range(48):
    layer_name = "transformer-h-" + str(i)
    layers.append(str(i))
    curr_layer_df = synt_df[synt_df["LayerName"] == layer_name]
    break

In [ ]:
closest_df

In [ ]:
#erdish...degree of separation

#contrastive

#wikidata

#occupation
#gender

In [ ]:
#two entity combined



# famous sports player